In [59]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers

In [60]:
# get data
dataset_path = keras.utils.get_file(
    "auto-mpg.data",
    "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
)

columns_name = [
    "MPG", "Cylinders", "Displacement", "Horsepower", "Weight",
    "Acceleration", "Model_Year", "Origin"
]

raw_dataset = pd.read_csv(
    dataset_path, 
    names=columns_name,
    sep=" ",
    comment="\t", 
    skipinitialspace=True,
    na_values="?"
)

print(raw_dataset.head())
dataset = raw_dataset.dropna()
origin = dataset.pop("Origin")

    MPG  Cylinders  Displacement  Horsepower  Weight  Acceleration  \
0  18.0          8         307.0       130.0  3504.0          12.0   
1  15.0          8         350.0       165.0  3693.0          11.5   
2  18.0          8         318.0       150.0  3436.0          11.0   
3  16.0          8         304.0       150.0  3433.0          12.0   
4  17.0          8         302.0       140.0  3449.0          10.5   

   Model_Year  Origin  
0          70       1  
1          70       1  
2          70       1  
3          70       1  
4          70       1  


In [61]:
encode_origin = tf.one_hot(origin, depth=4).numpy()
sub_dataset = pd.DataFrame(encode_origin[:, 1:], columns=["USA", "Europe", "Japan"])
dataset.index = sub_dataset.index
dataset=pd.concat([dataset, sub_dataset], axis=1)

dataset.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model_Year,USA,Europe,Japan
0,18.0,8,307.0,130.0,3504.0,12.0,70,1.0,0.0,0.0
1,15.0,8,350.0,165.0,3693.0,11.5,70,1.0,0.0,0.0
2,18.0,8,318.0,150.0,3436.0,11.0,70,1.0,0.0,0.0
3,16.0,8,304.0,150.0,3433.0,12.0,70,1.0,0.0,0.0
4,17.0,8,302.0,140.0,3449.0,10.5,70,1.0,0.0,0.0


In [62]:
x_train = dataset.sample(frac=0.8, random_state=0)
x_test = dataset.drop(x_train.index)

y_train = x_train.pop("MPG")
y_test = x_test.pop("MPG")

print(f"x_train: {x_train.shape}, x_test: {x_test.shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")

x_train: (314, 9), x_test: (78, 9)
y_train: (314,), y_test: (78,)


In [63]:
train_stats = x_train.describe().T

norm = lambda x: (x - train_stats["mean"]) / train_stats["std"]

normed_x_train = norm(x_train)
normed_x_test = norm(x_test)

In [64]:
train_db = tf.data.Dataset.from_tensor_slices(
    (normed_x_train.values, y_train.values)
)
train_db = train_db.shuffle(100).batch(32)
train_db

<BatchDataset shapes: ((None, 9), (None,)), types: (tf.float64, tf.float64)>

In [65]:
class Network(keras.Model):
    def __init__(self):
        super().__init__()
        self.layer_1 = layers.Dense(64, activation="relu")
        self.layer_2 = layers.Dense(64, activation="relu")
        self.output_layer = layers.Dense(1)

    def call(self, inputs, training=None, mask=None):
        layer1 = self.layer_1(inputs)
        layer2 = self.layer_2(layer1)
        output = self.output_layer(layer2)
        return output

In [66]:
model = Network()
model.build(input_shape=(None, 9))
model.summary()

Model: "network_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             multiple                  640       
_________________________________________________________________
dense_13 (Dense)             multiple                  4160      
_________________________________________________________________
dense_14 (Dense)             multiple                  65        
Total params: 4,865
Trainable params: 4,865
Non-trainable params: 0
_________________________________________________________________


In [70]:
optimzier = keras.optimizers.RMSprop(0.001)

for epoch in range(200):
    for step, (x, y) in enumerate(train_db):
        with tf.GradientTape() as tape:
            out = model(x)
            mse_loss = tf.reduce_mean(keras.losses.MSE(y, out))
            mae_loss = tf.reduce_mean(keras.losses.MAE(y, out))
        if step % 5 == 0:
            print(f"epoch: {epoch} -- step: {step} -- mse_loss: {mse_loss}")
            print("=" * 70)
        grads = tape.gradient(mse_loss, model.trainable_variables)
        optimzier.apply_gradients(zip(grads, model.trainable_variables))
        

epoch: 0 -- step: 0 -- mse_loss: 75.29277038574219
epoch: 0 -- step: 5 -- mse_loss: 53.45249938964844
epoch: 1 -- step: 0 -- mse_loss: 54.07611083984375
epoch: 1 -- step: 5 -- mse_loss: 55.3983268737793
epoch: 2 -- step: 0 -- mse_loss: 67.46318054199219
epoch: 2 -- step: 5 -- mse_loss: 74.72702026367188
epoch: 3 -- step: 0 -- mse_loss: 59.32172393798828
epoch: 3 -- step: 5 -- mse_loss: 66.74610900878906
epoch: 4 -- step: 0 -- mse_loss: 57.281585693359375
epoch: 4 -- step: 5 -- mse_loss: 57.17613983154297
epoch: 5 -- step: 0 -- mse_loss: 64.70547485351562
epoch: 5 -- step: 5 -- mse_loss: 59.26581573486328
epoch: 6 -- step: 0 -- mse_loss: 51.80408477783203
epoch: 6 -- step: 5 -- mse_loss: 59.18101501464844
epoch: 7 -- step: 0 -- mse_loss: 51.056678771972656
epoch: 7 -- step: 5 -- mse_loss: 55.68798065185547
epoch: 8 -- step: 0 -- mse_loss: 103.42742919921875
epoch: 8 -- step: 5 -- mse_loss: 53.1341438293457
epoch: 9 -- step: 0 -- mse_loss: 50.70848846435547
epoch: 9 -- step: 5 -- mse_los

[<tf.Variable 'dense_12/kernel:0' shape=(9, 64) dtype=float32, numpy=
 array([[-1.52300194e-01, -1.74181253e-01, -3.34539711e-01,
         -3.07454765e-01,  3.30851585e-01, -1.20471660e-02,
         -1.35033235e-01,  1.31627452e-02, -2.58101206e-02,
         -5.49390307e-03, -3.39912057e-01, -1.31741181e-01,
          2.70920724e-01, -1.42729133e-01, -1.11079045e-01,
          2.93168128e-01,  8.03056061e-02, -6.18683957e-02,
         -2.43860886e-01,  1.22649699e-01, -3.15426499e-01,
         -2.27682009e-01, -2.86196589e-01,  3.25116843e-01,
         -2.25095868e-01,  2.25060806e-02, -8.94799903e-02,
          1.79325297e-01, -1.87382042e-01,  1.34255618e-01,
          2.12203726e-01,  1.36059925e-01, -1.49603246e-03,
         -3.28183442e-01,  2.79951274e-01,  2.45501827e-02,
          1.32497296e-01, -3.30474645e-01, -1.66068360e-01,
          2.97869712e-01,  2.84900427e-01,  2.51890838e-01,
          1.88502267e-01,  2.27429047e-01, -1.78945914e-01,
          2.88927943e-01,  6.2